cannon
---
Following [tensorflow basics example](https://www.tensorflow.org/guide/basics)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import tensorflow as tf
import keras

In [2]:
speed1, azi1, pitch1 = 10, np.pi, np.pi/3
velocity = speed1*tf.Variable([
        np.cos(azi1)*np.cos(pitch1),
        np.sin(azi1)*np.cos(pitch1),
        np.sin(pitch1)
    ])
velocity

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([-5.00000000e+00,  6.12323400e-16,  8.66025404e+00])>

In [3]:
pi = np.pi

#simulates a ballistic trajectory and returns the smallest distance to the target position
#must be done with 
def simulate_trajectory_distance(speed, azi, pitch, target, origin = tf.constant([0.,0.,0.], dtype = 'float32'), timestep = 0.1, g = -10.):
    #get cartesian velocity from spherical coordinates
    velocity = speed*tf.Variable([
        tf.math.cos(azi)*tf.math.cos(pitch),
        tf.math.sin(azi)*tf.math.cos(pitch),
        tf.math.sin(pitch)
    ],
    dtype = 'float32')
#     print(f'Starting velocity {velocity}.')
    #make acceleration vector
    acc = tf.constant([0.,0., g])
    #record origin and first position after 1 timestep
    velocity_step = origin + velocity*timestep
    acceleration_step = (1/2)*acc*timestep**2
    history = [origin, origin+ velocity_step + acceleration_step]
    position = history[-1]
#     print(history)
    #record distances
    distances = [tf.norm(target - origin), tf.norm(target - position)]
    #while the ball is above the ground, timestep to change position
    while position[2] >= 0:
#         print(history[-1])
        position = 2*history[-1] - history[-2] + acc*(timestep**2)
        distances.append(tf.norm(target - position))
        history.append(position)
    return tf.math.reduce_min(tf.Variable(distances, dtype = 'float32'))
    

In [4]:
simulate_trajectory_distance(100,np.pi,np.pi/3, tf.Variable([100,10,200], dtype = 'float32')
                            )

<tf.Tensor: shape=(), dtype=float32, numpy=190.699>

In [26]:
%%time

seed = 2022
rng = np.random.default_rng(seed)

eps = np.finfo(np.float32).eps.item()

gamma = 0.99
rounds_per_load = 5
minimum_acceptable_score = 500
scores_to_average = 10
max_speed = 100
field_size = 2000

hit_distance = 10
miss_penalty = 500

num_inputs = 3
num_outputs = 3
num_hidden = 32

inputs = keras.layers.Input(shape = (num_inputs,))
dense_1 = keras.layers.Dense(num_hidden, activation = 'relu')(inputs)
outputs = keras.layers.Dense(num_outputs, activation = 'softmax')(dense_1)
model = keras.Model(inputs = inputs, outputs = outputs)

optimizer = tf.optimizers.Adam(learning_rate = 0.01)

avg_recent_score = 0
all_scores = [0]
total_loss = tf.constant(0.)

loss_fn = tf.losses.Huber()

model.trainable_variables[1] = tf.ones((32,1))

while avg_recent_score < minimum_acceptable_score:
    with tf.GradientTape() as tape:
        command_history = []
        loss_history = []
        for shot in range(rounds_per_load):
            target = field_size*rng.random(3) - field_size/2
            target[2] = abs(target[2])
            target = tf.convert_to_tensor(target, dtype = 'float32')
            target = tf.expand_dims(target, 0)
            
            command = model(target)
            speed, azi, pitch = command[0]
            speed = speed*max_speed
            azi = azi*2*pi
            pitch = pitch*pi
            
            loss = simulate_trajectory_distance(speed, azi, pitch, target, timestep = 0.5)
            loss_history.append(loss)
            
        total_loss = tf.constant(sum(loss_history))
        grads = tape.gradient(total_loss, model.trainable_variables)
        if grads == [None, None, None, None]:
            grads = tf.constant(
                [tf.ones(variables.shape, dtype = 'float32') for variables in model.trainable_variables],
                dtype = 'float32')
        print(grads)
        print(model.trainable_variables)
        print('optimizing')
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        all_scores.append(total_loss)
        loss_history.clear()
        
        avg_recent_score = np.mean(all_scores[-scores_to_average:])
        if len(all_scores[:-2])%1:
            print(f'Finished with trial {len(all_scores[:-2])}')

ValueError: TypeError: Scalar tensor has no `len()`
Traceback (most recent call last):

  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1170, in __len__
    raise TypeError("Scalar tensor has no `len()`")

TypeError: Scalar tensor has no `len()`

